In [2]:
pip install pillow

  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl (2.6 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.10.0 requires pyparsing>=2.3.1, which is not installed.
moviepy 2.1.2 requires python-dotenv>=0.10, which is not installed.
pyvista 0.44.2 requires scooby>=0.5.1, which is not installed.
moviepy 2.1.2 requires pillow<11.0,>=9.2.0, but you have pillow 11.1.0 which is incompatible.


In [6]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached charset_normalizer-3.4.1-cp311-cp311-win_am

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
genesis-world 0.2.1 requires OpenEXR, which is not installed.
genesis-world 0.2.1 requires pydantic==2.7.1, which is not installed.
genesis-world 0.2.1 requires pyglet, which is not installed.
genesis-world 0.2.1 requires PyOpenGL, which is not installed.
genesis-world 0.2.1 requires screeninfo, which is not installed.
moviepy 2.1.2 requires python-dotenv>=0.10, which is not installed.
mujoco 3.2.5 requires glfw, which is not installed.
mujoco 3.2.5 requires pyopengl, which is not installed.
pymeshlab 2023.12.post2 requires msvc-runtime, which is not installed.
pyvista 0.44.2 requires scooby>=0.5.1, which is not installed.
genesis-world 0.2.1 requires numpy==1.26.4, but you have numpy 2.1.3 which is incompatible.
moviepy 2.1.2 requires pillo

In [ ]:

import numpy as np
# import matplotlib.pyplot as plt
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam


In [4]:
pip install pyparsing

  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyvista 0.44.2 requires scooby>=0.5.1, which is not installed.


In [ ]:

# ----------------------

# ----------------------
class TrafficEnv:
    def __init__(self):
        self.phases = [0, 1, 2, 3]  # Four traffic light phase actions
        self.goal_durations = [20, 20, 35, 35]  # Optimal durations
        self.max_duration = 50
        self.state = np.random.randint(5, 50, size=4)  # Initial random durations for each phase
        self.action_space = len(self.phases)
        self.observation_space = len(self.state)

    def step(self, action):
        # Action: increment or decrement duration of selected phase to get closer to optimal
        delta = np.random.choice([-1, 1])  # Adjust duration randomly up/down
        self.state[action] = np.clip(self.state[action] + delta, 5, self.max_duration)

        # Reward is negative sum of distance from goal durations
        distance = np.abs(np.array(self.state) - np.array(self.goal_durations))
        reward = -np.sum(distance)

        done = np.all(distance <= 1)  # Done if all durations match goal
        return np.array(self.state), reward, done

    def reset(self):
        self.state = np.random.randint(5, 50, size=4)
        return np.array(self.state)

# ----------------------
# DQN Agent Definition
# ----------------------
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount rate
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Network for Deep-Q learning
        model = Sequential([
            Input(shape=(self.state_size,)),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state[np.newaxis], verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in minibatch:
            target = reward + self.gamma * np.amax(self.model.predict(next_state[np.newaxis], verbose=0)[0])
            target_f = self.model.predict(state[np.newaxis], verbose=0)
            target_f[0][action] = target
            self.model.fit(state[np.newaxis], target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def get_q_values(self, state):
        return self.model.predict(state[np.newaxis], verbose=0)

# ----------------------
# Training
# ----------------------

env = TrafficEnv()
state_size = env.observation_space
action_size = env.action_space
agent = DQNAgent(state_size, action_size)

EPISODES = 500
q_convergence = []
phase_tracking = []

for e in range(EPISODES):
    state = env.reset()
    total_reward = 0
    for time in range(50):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state)
        state = next_state
        total_reward += reward
        if done:
            break
    agent.replay()
    q_val = np.mean(agent.get_q_values(state))
    q_convergence.append(q_val)
    phase_tracking.append(state.copy())

    if e % 50 == 0:
        print(f"Episode {e}/{EPISODES}, Reward: {total_reward}, Q-Value: {q_val:.2f}, Epsilon: {agent.epsilon:.2f}, Durations: {state}")

# ----------------------
# Plotting Results
# ----------------------
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(q_convergence)
plt.title("Q-Value Convergence")
plt.xlabel("Episode")
plt.ylabel("Mean Q-Value")

plt.subplot(1, 2, 2)
phase_tracking = np.array(phase_tracking)
for i in range(4):
    plt.plot(phase_tracking[:, i], label=f"Phase {i+1}")
plt.axhline(y=20, color='g', linestyle='--', label='Target 20')
plt.axhline(y=35, color='r', linestyle='--', label='Target 35')
plt.title("Phase Duration Convergence")
plt.xlabel("Episode")
plt.ylabel("Duration")
plt.legend()
plt.tight_layout()
plt.show()

# Verify final Q-values
final_state = env.reset()
final_qs = agent.get_q_values(final_state)
print("\nFinal Q-values for state", final_state, ":", final_qs)

ModuleNotFoundError: No module named 'PIL'